In [ ]:
# !pip install google-cloud-aiplatform[prediction]>=1.16.0 fastapi nvtabular git+https://github.com/NVIDIA-Merlin/models.git --user

# Buidling a custom Vertex AI endpoint for Merlin Query Tower

**IMPORTANT** Make sure you are running this notebook in a DLVM (e.g. tensorflow enterprise 2.8) to build the image

________
**This will not work in the training container**
________

Your output should look like this - you are going to use the query model endpoint to create a custom container

![](img/merlin-bucket.png)


In [1]:
from datetime import datetime


PROJECT = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
REGION = 'us-central1' 
BUCKET = 'gs://spotify-beam-v3'
REPOSITORY = 'merlin-spotify-cpr'
ARTIFACT_URI = f'{BUCKET}/merlin-processed'
MODEL_DIR = f'{BUCKET}/merlin-processed/query_model_merlin'
PREFIX = 'merlin-spotify'

In [2]:
# !gsutil mb -l us-central1 gs://wortz-project-bucket

In [3]:
# !echo y | docker container prune
# !echo y | docker image prune

### Set up repo and configure Docker (one-time)

In [4]:
# Create the repo if needed for the artifacts

! gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [5]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [6]:
! rm -rf app
! mkdir app

### Dependency file
The first few are for the server handling traffic
Nvtabular was downgraded for this example, it may not be necessary in future versions

In [40]:
%%writefile app/requirements.txt
uvicorn[standard]==0.15.0
gunicorn==20.1.0
fastapi==0.68.1
google-cloud-aiplatform
merlin-models
nvtabular
gcsfs
google-cloud-storage

Overwriting app/requirements.txt


### Predictor module and class
This is an adaptation of the CPR examples

This was locally tested and created shortly after model creation in the [tensorflow-predict](tensorflow-predict.ipynb) notebook

In [41]:
%%writefile app/dataset_to_tensors.py

try:
    import cudf
except ImportError:
    cudf = None
import pandas as pd
import tensorflow as tf
from typing import Dict
from merlin.io import Dataset
import itertools


def cupy_array_to_tensor(array):
    return tf.experimental.dlpack.from_dlpack(array.reshape(-1, 1).toDlpack())

def numpy_array_to_tensor(array):
    return tf.convert_to_tensor(array.reshape(-1, 1))

def cudf_series_to_tensor(col) -> tf.Tensor:
    "Convert a cudf.Series to a TensorFlow Tensor with DLPack"
    if isinstance(col.dtype, cudf.ListDtype):
        values = col.list.leaves.values
        offsets = col.list._column.offsets.values
        row_lengths = offsets[1:] - offsets[:-1]
        return cupy_array_to_tensor(values), cupy_array_to_tensor(row_lengths)
    else:
        return cupy_array_to_tensor(col.values)

def pandas_series_to_tensor(col) -> tf.Tensor:
    if len(col) and pd.api.types.is_list_like(col.values[0]):
        values = pd.Series(itertools.chain(*col)).values
        row_lengths = col.map(len).values
        return numpy_array_to_tensor(values), numpy_array_to_tensor(row_lengths)
    else:
        return numpy_array_to_tensor(col.values)
        
    
def dataset_to_tensors(dataset: Dataset) -> Dict[str, tf.Tensor]:
    """Convert a DataFrame to Dict of Tensors"""
    df = dataset.to_ddf().compute()
    if isinstance(df, pd.DataFrame):
        col_to_tensor = pandas_series_to_tensor
    else:
        col_to_tensor = cudf_series_to_tensor
    return {
        column: col_to_tensor(df[column])
        for column in df.columns
    }

Overwriting app/dataset_to_tensors.py


In [42]:
%%writefile app/predictor.py

import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf
import logging
from dataset_to_tensors import *
import time


# These are helper functions that ensure the dictionary input is in a certain order and types are preserved
# this is to get scalar values to appear first in the dict to not confuse pandas with lists https://github.com/pandas-dev/pandas/issues/46092
reordered_keys = ['collaborative', 'album_name_pl', 'artist_genres_pl', 
                  'artist_name_pl', 'artist_pop_can', 'description_pl', 
                  'duration_ms_songs_pl', 'n_songs_pl', 'name', 'num_albums_pl', 
                  'num_artists_pl', 'track_name_pl', 'track_pop_pl', 
                  'duration_ms_seed_pl', 'pid', 'track_uri_pl']  

float_num_fix = ['n_songs_pl','num_albums_pl','num_artists_pl','duration_ms_seed_pl']
float_list_fix = ['track_pop_pl', 'duration_ms_songs_pl']
    
def fix_list_num_dtypes(num_list):
    "this fixes lists of ints to list of floats converted in json input"
    return [float(x) for x in num_list]

def fix_num_dtypes(num):
    "this fixes ints and casts to floats"
    return float(num)

def fix_types(k, v):
    if k in float_num_fix:
        return fix_num_dtypes(v)
    if k in float_list_fix:
        return fix_list_num_dtypes(v)
    else:
        return v

def create_pandas_instance(inputs):
    """
    Helper function to reorder the input to have a sclar first for pandas
    And fix the types converted when data is imported by fastAPI
    """
    if type(inputs) == list:
        header = inputs[0]
        reordered_header_dict = {k: fix_types(k,header[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_header_dict, orient='index').T
        if len(inputs) > 1:
            for ti in inputs[1:]:
                reordered_dict = {k: fix_types(k,ti[k]) for k in reordered_keys}
                pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(reordered_dict, orient='index').T)
    else:
        reordered_dict = {k: fix_types(k,inputs[k]) for k in reordered_keys}
        pandas_instance = pd.DataFrame.from_dict(reordered_dict, orient='index').T
    return pandas_instance

class Predictor():
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri):
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        logging.info("loading model and workflow")
        start_init = time.process_time()
        
        #test_bucket = 'gs://jt-merlin-scaling'
        self.model = tf.keras.models.load_model(os.path.join(artifacts_uri, "query_model_merlin" ))
        # self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow")) # TODO: parameterize
        self.workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow"))
        # self.workflow = nvt.Workflow.load('gs://jt-merlin-scaling/nvt-last5-v1full/nvt-analyzed') # TODO: parametrize
        self.workflow = self.workflow.remove_inputs(
            [
                'track_pop_can', 
                'track_uri_can', 
                'duration_ms_can', 
                'track_name_can', 
                'artist_name_can',
                'album_name_can',
                'album_uri_can',
                'artist_followers_can', 
                'artist_genres_can',
                'artist_name_can', 
                'artist_pop_can',
                'artist_pop_pl',
                'artist_uri_can', 
                'artists_followers_pl'
            ]
        )
        return self
        
    def predict(self, prediction_input):
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        # handle different input types, can take a dict or list of dicts
        self.n_rows = len(prediction_input)
        start = time.process_time()
        pandas_instance = create_pandas_instance(prediction_input)
        #logging.info(f"Pandas conversion took {time.process_time() - start} seconds")
        print(f"Pandas conversion took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_inputs = nvt.Dataset(pandas_instance)
        #logging.info(f"NVT data loading took {time.process_time() - start} seconds")
        print(f"NVT data loading took {time.process_time() - start} seconds")
        start = time.process_time()
        transformed_instance = self.workflow.transform(transformed_inputs)
        print(f"Workflow transformation took {time.process_time() - start} seconds")

        # def predict(self, instances):
        start = time.process_time()
        
        batch = dataset_to_tensors(transformed_instance)
        print(f"converting to dict_tensors took {time.process_time() - start} seconds")
        start = time.process_time()
        output = self.model(batch)
        print(f"Generating query embeddings took {time.process_time() - start} seconds")
        return transformed_instance, output, batch

Overwriting app/predictor.py


In [43]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import json
import numpy as np
import os
import logging


from google.cloud import storage
from predictor import Predictor

app = FastAPI()

predictor_instance = Predictor()
loaded_predictor = predictor_instance.load(artifacts_uri = os.environ['AIP_STORAGE_URI'])

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    preprocessed_inputs = loaded_predictor.preprocess(instances)
    outputs = loaded_predictor.predict(preprocessed_inputs)

    return {"predictions": outputs.numpy().tolist()}

Overwriting app/main.py


In [44]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Overwriting app/prestart.sh


In [45]:
%%writefile app/instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Overwriting app/instances.json


In [46]:
#make it a package
!touch app/__init__.py

In [47]:
%%writefile Dockerfile

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.12
WORKDIR /app 

COPY ./app/requirements.txt /requirements.txt
RUN pip install -r /requirements.txt

COPY ./app /app
EXPOSE 80
    
CMD ["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

Overwriting Dockerfile


In [48]:
SERVER_IMAGE = "merlin-prediction-cpr"  # @param {type:"string"} 
REMOTE_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{SERVER_IMAGE}"

!docker build -t $REMOTE_IMAGE_NAME .

Sending build context to Docker daemon  108.7MB
Step 1/7 : FROM nvcr.io/nvidia/merlin/merlin-tensorflow:22.12
 ---> 58c045557af1
Step 2/7 : WORKDIR /app
 ---> Using cache
 ---> c07e8037ff83
Step 3/7 : COPY ./app/requirements.txt /requirements.txt
 ---> 50c2f8f08a1c
Step 4/7 : RUN pip install -r /requirements.txt
 ---> Running in aa08a78adce9
ERROR: dask-cudf 22.8.0a0+304.g6ca81bbc78.dirty requires cupy-cuda118<12,>=9.5.0, which is not installed.
ERROR: cudf 22.8.0a0+304.g6ca81bbc78.dirty requires cupy-cuda118<12,>=9.5.0, which is not installed.
ERROR: tensorflow 2.10.0+nv22.11 has requirement flatbuffers>=2.0, but you'll have flatbuffers 1.12 which is incompatible.
ERROR: tensorflow 2.10.0+nv22.11 has requirement tensorboard<2.11,>=2.10, but you'll have tensorboard 2.9.1 which is incompatible.
ERROR: tensorflow 2.10.0+nv22.11 has requirement tensorflow-estimator<2.11,>=2.10.0, but you'll have tensorflow-estimator 2.9.0 which is incompatible.
ERROR: merlin-core 0.10.0 has requirement fs

#### If you are debugging, be sure to set `-d` detached flag off and run the commands in console

### Copy/paste if you want to run from console for testing
```python
docker run --gpus all -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-beam-v3/merlin-processed \
            us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr
```

##### No GPU:
```python
docker run -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-beam-v3/merlin-processed \
            us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr
```

## Stop the images if they are running

In [49]:
! docker stop $SERVER_IMAGE
! docker rm $SERVER_IMAGE

merlin-prediction-cpr
merlin-prediction-cpr


#### Test the health route

In [51]:
! curl localhost/health

curl: (56) Recv failure: Connection reset by peer


In [23]:
## Ground truth candidate:
    # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
    # 'artist_name_can': 'Russ', 
    # 'track_name_can': 'We Just Havent Met Yet', 
## TODO - we have to overload with candidate data because of the workflow transform, add overloaded values in the predictor
TEST_INSTANCE = {'collaborative': 'false',
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 'artist_pop_can': 82.0, 
                 'description_pl': '', 
                 'duration_ms_songs_pl': [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], 
                 'n_songs_pl': 8.0, 
                 'name': 'Lit Tunes ', 
                 'num_albums_pl': 8.0, 
                 'num_artists_pl': 8.0, 
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 'track_pop_pl': [79.0, 58.0, 83.0, 71.0, 57.0],
                 'duration_ms_seed_pl': 51023.1,
                 'pid': 1,
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt']
                     }

In [37]:
import json
import requests

json_instance = json.dumps({"instances": [TEST_INSTANCE]})
print(json_instance)

{"instances": [{"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "description_pl": "", "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "sp

### Test the predict route

In [40]:
%%timeit
requests.post('http://localhost/predict', data=json_instance).json()

3.19 s ± 509 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Push the container once ready and testing is complete

In [80]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr]

df02f55f: Preparing 
5f06f5f8: Preparing 
8062ff83: Preparing 
3dceb6e0: Preparing 
3aa55fbd: Preparing 
9c500afd: Preparing 
ca29c9f2: Preparing 
c8057c95: Preparing 
555d3a5b: Preparing 
7cf26842: Preparing 
9b7c9426: Preparing 
b850bdf8: Preparing 
12b87e2d: Preparing 
d4c011cd: Preparing 
32bae622: Preparing 
7dce553c: Preparing 
fd489635: Preparing 
47e88671: Preparing 
08bc66d2: Preparing 
02ddb49b: Preparing 
40bc31be: Preparing 
3d22fc2d: Preparing 
d03b40d8: Preparing 
4453f24a: Preparing 
b1d8d420: Preparing 
80eb8994: Preparing 
49e0fefd: Preparing 
9a0c8694: Preparing 
2f6543d2: Preparing 
58199899: Preparing 
faf5524c: Preparing 
9dea71d9: Preparing 
b4fa6312: Preparing 
c5236059: Preparing 
fc353e53: Preparing 
e1954466: Preparing 
0d3a32ca: Preparing 
9b216faa: Preparing 
3f10d323: Preparing 
452845b7: Preparing 
0b8eda77: Preparing

### Deploy to Vertex AI

After the serving metadata is set below, the model is properly abstracted for use on Vertex

In [81]:
MODEL_DISPLAY_NAME = "Merlin Spotify Query Tower Model"
from google.cloud import aiplatform

model = aiplatform.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        artifact_uri=ARTIFACT_URI,
        serving_container_image_uri=REMOTE_IMAGE_NAME,
        serving_container_predict_route='/predict',
        serving_container_health_route='/health',
        serving_container_command=["sh", "-c", "uvicorn main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"],
        serving_container_args='--gpus all',
        sync=True,
    )

Creating Model
Create Model backing LRO: projects/934903580331/locations/us-central1/models/5656727840163364864/operations/2907409254111510528
Model created. Resource name: projects/934903580331/locations/us-central1/models/5656727840163364864@1
To use this Model in another session:
model = aiplatform.Model('projects/934903580331/locations/us-central1/models/5656727840163364864@1')


In [82]:
endpoint = model.deploy(machine_type="n1-standard-4",
                        accelerator_type="NVIDIA_TESLA_T4",
                        accelerator_count=1)

Creating Endpoint
Create Endpoint backing LRO: projects/934903580331/locations/us-central1/endpoints/433264755948388352/operations/646602241171521536
Endpoint created. Resource name: projects/934903580331/locations/us-central1/endpoints/433264755948388352
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/433264755948388352')
Deploying model to Endpoint : projects/934903580331/locations/us-central1/endpoints/433264755948388352
Deploy Endpoint model backing LRO: projects/934903580331/locations/us-central1/endpoints/433264755948388352/operations/7631685263223160832
Endpoint model deployed. Resource name: projects/934903580331/locations/us-central1/endpoints/433264755948388352


In [85]:
#make a prediction

endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])

Prediction(predictions=[[1.391043424606323, 0.0, 1.188787579536438, 0.4119991064071655, 0.0, 0.0, 0.4376969039440155, 0.2364768087863922, 1.033334255218506, 0.8548258543014526, 0.0, 0.4630281031131744, 0.3279293179512024, 0.0, 1.203551054000854, 0.5297332406044006, 0.0, 0.0, 0.3997420370578766, 0.3299564123153687, 2.238887786865234, 0.0, 1.095603108406067, 0.0, 0.0, 1.459867835044861, 0.0, 0.0, 0.0, 1.231191158294678, 0.5454803705215454, 0.0, 0.0, 0.0, 1.907413601875305, 0.6069029569625854, 1.359937787055969, 0.0, 0.529766857624054, 1.814321875572205, 0.1255402266979218, 1.029636263847351, 0.0, 1.71973180770874, 0.0, 0.0, 0.1176593527197838, 0.0, 0.0, 0.9931750297546387, 0.0, 0.0, 0.9380308985710144, 0.1176023855805397, 4.363896369934082, 0.112496443092823, 1.584652543067932, 2.07372522354126, 1.642884135246277, 0.0, 0.0, 0.01452790945768356, 0.0, 0.1659038364887238, 0.0, 2.377683877944946, 1.798207402229309, 0.0, 0.8842217326164246, 0.05270992964506149, 1.103273868560791, 0.1857113689

## Finished - now go on to the next notebook to create a [matching engine notebook](03-matching-engine.ipynb) and test out the first end to end recommendation

Be sure to use the output of the endpoint logs above to save the endpoint for use in the matching engine notebook

e.g.:

```python
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/934903580331/locations/us-central1/endpoints/494907775848022016')
```

### Timing analysis

In [51]:
%%timeit
endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])

3.9 s ± 206 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [86]:
%%timeit
endpoint.predict(instances=[[TEST_INSTANCE, TEST_INSTANCE]])

4 s ± 176 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
